Classification Schoolwork

@author：刘士坤 2016011371

9个类别的高光谱数据分类,使用SVM分类

导入相应的库

In [1]:
import os
import scipy.io as sio
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

### 1.将高光谱数据train导入，并按照8 : 2的比例划分成训练集与验证集。

数据集train导入形成类别标签

In [2]:
#使用字典存储所有数据
data_dict = {}
data_path = "./data/train"
for i in os.listdir(data_path):
    m = sio.loadmat(os.path.join(data_path,i))
    data_dict[i.split(".")[0]] = m[i.split(".")[0]]

In [3]:
#data存储
data = []
for i in data_dict.keys():
    for j in data_dict[i]:
        data.append(j)
data = np.array(data)
data

array([[3693, 4256, 4503, ..., 1050, 1024, 1024],
       [2589, 4372, 4334, ..., 1050, 1024, 1019],
       [3693, 4128, 4507, ..., 1045, 1029, 1019],
       ...,
       [3192, 4235, 4313, ..., 1026, 1023, 1000],
       [3192, 4111, 4313, ..., 1030, 1014, 1008],
       [3192, 4115, 4313, ..., 1031, 1014, 1013]], dtype=uint16)

In [4]:
#label存储
label=[]
for i in data_dict.keys():
    y = re.sub("\D","",i)
    for j in range(len(data_dict[i])):
        label.append(y)
label = np.array(label,dtype=np.int64)

In [5]:
label

array([10, 10, 10, ...,  8,  8,  8], dtype=int64)

按照8 : 2的比例划分成训练集与验证集

In [6]:
X_train,X_valid,y_train,y_valid = train_test_split(data,label,test_size=0.2,shuffle=True)

### 2.构建分类器模型

数据预处理

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

D:\Users\lsk\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


svc分类器

In [8]:
svc = SVC(C=19,gamma=0.01,class_weight='balanced')
svc.fit(X_train,y_train)

SVC(C=19, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
y_pred = svc.predict(X_valid)
print(accuracy_score(y_valid,y_pred))

0.948014440433213


### 3.测试集

将高光谱数据测试集test导入

In [10]:
test_data = np.array(sio.loadmat("./data/test/data_test_final.mat")['data_test_final'], dtype=np.float64)

In [11]:
x_test = scaler.transform(test_data)

In [12]:
y_test = svc.predict(x_test)

导出标签数据

In [13]:
data = pd.DataFrame(y_test)
data.to_csv("./data/test.csv")